In [1]:
pip install --upgrade tinyautoml

Note: you may need to restart the kernel to use updated packages.


In [3]:
import TinyAutoML
dir(TinyAutoML)

['Path',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'os']

In [2]:
import os
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime
from tqdm import tqdm


if not "root" in locals():
    current_path = Path(os.getcwd())
    root = current_path.parent.absolute()
os.chdir(root)
    
from tools.add_AR_cols import add_AR_cols
from data_treatment.box_creator import create_binary_box
import TinyAutoML as tam


In [3]:
vxx = pd.read_csv("VXX.csv")
df = pd.read_csv('database.csv')


In [3]:
vxx

,Date,Open,High,Low,Close,Adj Close,Volume
0,2018-01-25,110.639999,110.639999,110.639999,110.639999,110.639999,0
1,2018-01-26,110.639999,110.639999,110.639999,110.639999,110.639999,0
2,2018-01-29,116.800003,118.320000,116.800003,118.320000,118.320000,350
3,2018-01-30,123.160004,127.239998,121.959999,122.199997,122.199997,15825
4,2018-01-31,119.839996,122.720001,119.839996,122.599998,122.599998,2300
...,...,...,...,...,...,...,...
1027,2022-02-23,22.730000,24.770000,22.650000,24.700001,24.700001,76133500
1028,2022-02-24,27.400000,27.450001,23.770000,23.924999,23.924999,110653700
1029,2022-02-25,23.719999,24.200001,22.620001,22.950001,22.950001,65153500
1030,2022-02-28,24.700001,24.969999,23.549999,24.000000,24.000000,74178800


In [4]:
vxx_dates = [datetime.strptime(d, "%Y-%m-%d").date() for d in vxx["Date"]][:600]
try:
    df['Date'] = pd.to_datetime(df['Date']).dt.date
    df = create_binary_box(df, relative_threshold = 0.05, box_length=2).set_index(['Date']).dropna(axis = 0)
except:
    pass

In [5]:

starting_cash = 1000
cash = starting_cash
n_long = 0
max_exposition = 0.5
pct_fees = 0
base_fees = 0

cash_list = []
assets_list = []
portfolio_list = []
dates = []
vxx_values = []

for i in tqdm(range(len(vxx_dates))):
    price = vxx["Close"][i]
    if i % 2 == 0:
        date = vxx_dates[i]
        
        if i % 50 == 0:
            X,y = df.drop('Box', axis=1)[df.index<date], df['Box'][df.index<date]
            mpipe = tam.Estimator.MetaPipeline()
            mpipe.fit(X,y, grid_search=True)
        X_today = df.drop('Box', axis=1)[df.index <= date]
        result = mpipe.predict(X_today)[-1]
        if result == 1:
            if max_exposition * cash > n_long * price:
                n_long += 1
                cash -= (price * (1 + pct_fees) - base_fees)
        else:
            cash += n_long * price
            n_long = 0
        dates.append(date)
        vxx_values.append(price)
        vxx_values[-1] = vxx_values[-1]/vxx_values[0] * starting_cash
        cash_list.append(cash)
        assets_list.append(n_long * price)
        portfolio_list.append(cash + n_long * price)
        print("Current portfolio value : ", portfolio_list[-1])

  0%|          | 0/600 [00:00<?, ?it/s]


AttributeError: module 'TinyAutoML' has no attribute 'Estimator'

In [23]:
import plotly.express as px

result_df = pd.DataFrame({"Date" : dates, "Portfolio value" : portfolio_list, "Cash" : cash_list, "Assets" : vxx_list})

In [28]:
fig = px.line(result_df, x = "Date", y = ["Portfolio value", "Cash", "Assets"])
with open("output_trading.html", "w") as f:
    f.write(fig.to_html())

In [20]:
len(vxx_list)

0

In [7]:
tam.Estimator

AttributeError: module 'TinyAutoML' has no attribute 'Estimator'